## Fetch earnings date from JPX website

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
from sqlalchemy import create_engine

In [3]:
JPX_URL = 'https://www.jpx.co.jp/listing/event-schedules/financial-announcement/index.html'
DATABASE_URL='postgresql://@localhost:5432/kabu_db_local'

In [4]:
engine = create_engine(DATABASE_URL)

In [5]:
response = requests.get(JPX_URL)
soup = BeautifulSoup(response.text, 'lxml')

In [6]:
xlses = []
for a in soup.find_all('a', href=True):
    if a['href'].endswith('.xls'):
        xlses.append(a['href'])

In [7]:
COLUMN_MAPPING = {'発表予定日':'date', 'コード':'code', '会社名':'name', '決算期末':'term', '業種名':'segment', '種別':'pattern', '市場区分':'market'}

In [8]:
PATTERN_MAPPING = {
    '第３四半期':'3Q', '第２四半期':'2Q', '第１四半期':'1Q', '本決算':'4Q', '-':''
}

In [9]:
def download_xls(file_name, url):
    res = requests.get(url)
    save_to = '{}{}'.format('./', file_name)
    if res.status_code == 200:
        open(save_to, 'wb').write(res.content)
        print('Done')
        return save_to

In [10]:
def clean_dataframe(df):
    df = pd.read_excel(file_path, skiprows=2)
    df = df.dropna()
    df = df.rename(columns=COLUMN_MAPPING)
    df['pattern'] = df['pattern'].map(PATTERN_MAPPING)
    df['code']=df['code'].astype(int)
    df['code']=df['code'].astype(str)
    return df
    

In [11]:
file_paths = []
for idx, xls in enumerate(xlses):
    file_name = xls.split('/')[-1]
    path = download_xls(file_name, '{}{}'.format('https://www.jpx.co.jp', xls))
    file_paths.append(path)

Done
Done


In [12]:
dfs = []
for file_path in file_paths:
    idx_key = file_path.split('/')[-1].replace('.xls', '')
    df = pd.read_excel(file_path, skiprows=2)
    df = clean_dataframe(df)
    df['id']=idx_key+'-'+df['code']
    dfs.append(df)

In [13]:
combined_df = pd.concat(dfs)

In [14]:
combined_df['date'] = combined_df['date'].replace('未定', '')

In [15]:
combined_df = combined_df.set_index('id')

In [16]:
combined_df['date'] = pd.to_datetime(combined_df['date'])

In [17]:
combined_df['date'] = combined_df.date.astype(object).where(combined_df.date.notnull(), None)

In [18]:
combined_df

,date,code,name,term,segment,pattern,market
id,,,,,,,
kessan05_0701-2164,2020-07-01 00:00:00,2164,地域新聞社,8月31日,サービス業,3Q,ジャスダック
kessan05_0701-2809,2020-07-02 00:00:00,2809,キユーピー,11月30日,食料品,2Q,一部
kessan05_0701-2830,2020-07-02 00:00:00,2830,アヲハタ,11月30日,食料品,2Q,二部
kessan05_0701-2918,2020-07-02 00:00:00,2918,わらべや日洋ホールディングス,2月末日,食料品,1Q,一部
kessan05_0701-3549,2020-07-02 00:00:00,3549,クスリのアオキホールディングス,5月20日,小売業,4Q,一部
...,...,...,...,...,...,...,...
kessan06_0701-9810,None,9810,日鉄物産,3月31日,卸売業,1Q,一部
kessan06_0701-9900,None,9900,サガミホールディングス,3月31日,小売業,1Q,一部
kessan06_0701-9969,None,9969,ショクブン,3月31日,小売業,1Q,二部


In [19]:
combined_df.to_sql('earnings_schedule', engine, if_exists='replace')